In [17]:
DB_PATH = './data/dbpedia/db_part/'
# DB_PATH = './data/dbpedia_t2ksubset/db/'
import trident
# trident.setLoggingLevel(0)
g = trident.Db(DB_PATH)
type_id = g.lookup_id('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>')
label_id = g.lookup_id('<http://www.w3.org/2000/01/rdf-schema#label>')
assert label_id != None
print('Trident KB has %d terms, %d triples' % (g.n_terms(), g.n_triples()))

Trident KB has 31988193 terms, 108857097 triples


In [18]:
import sys, urllib.parse
REDIR = 'data/transitive_redirects_en.ttl'
redir = {}
for i,line in enumerate(open(REDIR)):
    source, _, target = line.strip()[:-2].split(None, 2)
    source, target = urllib.parse.unquote(source[1:-1]), urllib.parse.unquote(target[1:-1])
    redir[source] = target
    if 0 == i % 100000:
        print('Loading redirects line %d    ' % i, end='\r', file=sys.stderr)

In [19]:
uri = 'http://dbpedia.org/resource/Holmes_Regional_Medical_Center'
g.lookup_id('<%s>'%redir.get(uri, uri))

22599749

In [204]:
def datatype(node_id):
    # RDF datatype
    node = g.lookup_str(node_id)
    if not node:
        return '', ''
    elif node[-1] == '>':
        if node[0] == '<':
            return node[1:-1], 'uri' # uri
        elif node[0] == '"' and '^^' in node:
            lit, dtype = node.rsplit('^^', 1)
            return lit[1:-1], dtype[1:-1] # typed literal
    elif node[0] == '"':
        if node[-1] == '"':
            return node[1:-1], 'str' # string
        elif '@' in node:
            lit, lang = node.rsplit('@', 1)
            return lit[1:-1], '@%s'%lang # language string
def strs(i):
    ss = set()
    # Entity strings
    node, dtype = datatype(i)
    if dtype == 'uri':
        for l in g.o(i, label_id):
            s = datatype(l)[0]
            ss.update( set([s]) )
    elif dtype == 'str' or dtype.startswith('@'):
        s = node
        ss.update( set([s]) )
    return ss

In [285]:
import os, collections
t2k_dir = './data_t2k/output-v1-goldCandidates/'
missing = set([row[0] for row in csv.reader(open(os.path.join(t2k_dir, 'missing_gs_instance.csv')))])
print('T2K says that %d uris are missing' % len(missing))
really_missing = set([uri for uri in missing if g.lookup_id('<%s>'%uri) == None])
print('I think that %d uris are missing' % len(really_missing))

missing_ids = set([g.lookup_id('<%s>'%uri) for uri in missing if g.lookup_id('<%s>'%uri)])
missing_types = [g.lookup_str(t) for i in missing_ids for t in g.o(i, type_id)]
missing_types = [t for t in missing_types if 'http://dbpedia.org/ontology/' in t]
# collections.Counter(missing_types).most_common()

T2K says that 1428 uris are missing
I think that 32 uris are missing


In [289]:
import os, glob, itertools, csv
table_fnames = glob.glob('data_t2k/db/dbpedia/*.csv')
table_headers = {os.path.basename(fname)[:-4]:itertools.islice(open(fname), 3) for fname in table_fnames}
table_headers = {key: list(zip(*csv.reader(val))) for key, val in table_headers.items()}

def get_table_row(i, attributes):
    for att, uri, typ in attributes:
        print(att)
        p = g.lookup_id('<%s>'%uri)
        if p:
            for o in g.o(i, p):
                yield att, g.lookup_str(o)

missing_types_ids = set()
ok=False
for i in missing_ids:
#     print(g.lookup_str(i))
    types_missing = True
    for t in g.o(i, type_id):
        t_uri = g.lookup_str(t)
        if 'http://dbpedia.org/ontology/' in t_uri:
            t_name = t_uri[1:-1].replace('http://dbpedia.org/ontology/','')
            if t_name in table_headers:
                types_missing = False
#             print('\t', t_name, t_name in table_headers)
#             for att,o in get_table_row(i, table_headers.get(t_name, [])):
#                 print(att,o)
#                 ok=True
    if types_missing:
        missing_types_ids.add(i)
    if ok:
        break

print('There are %d entities that cannot be added to any table' % len(missing_types_ids))
missing_types = [g.lookup_str(t) for i in missing_types_ids for t in g.o(i, type_id)]
missing_types = [t for t in missing_types if 'http://dbpedia.org/ontology/' in t]
# collections.Counter(missing_types).most_common()

# set([g.lookup_str(m) for m in missing_ids - missing_types_ids ])

There are 1242 entities that cannot be added to any table


In [205]:
uri = 'http://dbpedia.org/resource/Catholic_Democrats_(The_National_Party)'
uri = 'http://dbpedia.org/resource/Christian_Democrats_(The_National_Party)'
# uri = 'http://dbpedia.org/resource/Christian_Solidarity_Party'
uri = 'http://dbpedia.org/resource/Prince_Charles_Hospital'
i = g.lookup_id('<%s>'%uri)
print('entity %d has %d outlinks and %d inlinks' % (i, len(g.po(i)), len(g.ps(i))))
for p,o in g.po(i):
    print('%50s %-50s' % (g.lookup_str(p), g.lookup_str(o)))
#     print(p == label_id)
    ss = strs(o)
    if ss:
        print('%-50s'%g.lookup_str(p), *ss)
#     print()

entity 27324403 has 6 outlinks and 0 inlinks
                <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:1978_establishments_in_the_United_Kingdom>
                <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Hospital_buildings_completed_in_1978>
                <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Hospitals_established_in_1978>
                <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Hospitals_in_Merthyr_Tydfil>
                <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:NHS_Wales_hospitals>
      <http://www.w3.org/2000/01/rdf-schema#label> "Prince Charles Hospital"@en                      
<http://www.w3.org/2000/01/rdf-schema#label>       Prince Charles Hospital
